In [ ]:
#FOR DEENA
!pip3 install sagemaker
!pip3 install boto3
!pip3 install json
!pip3 install transformer

#need to request more quotas for "ml.m5.xlarge for transform job usage", "ml.m5.xlarge for endpoint usage", 

In [1]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hugging Face Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID': 'mdarhri00/named-entity-recognition',
    'HF_TASK': 'token-classification'
}

# Create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.26.0',
    pytorch_version='1.13.1',
    py_version='py39',
    env=hub,
    role=role,
)

# Deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,  # Number of instances
    instance_type='ml.m5.xlarge'  # EC2 instance type
)

# Make a prediction using the deployed model
predictor.predict({
    "inputs": "My name is Sarah Jessica Parker but you can call me Jessica and I live in College Park",
})

#choppy results so could make a code that if entity is the same for two things with neighboring indexes, could that as one location or person name

#NOTE for error regarding (ResourceLimitExceeded) you have to go to SageMaker and search "increase quota" then search "ml.m5.xlarge" and request more quotas each time. Then re-run the code.

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2024-01-10-17-43-22-630
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-inference-2024-01-10-17-43-23-210
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-inference-2024-01-10-17-43-23-210


-----!

[{'entity': 'Person_Name',
  'score': 0.8258283734321594,
  'index': 4,
  'word': 'Sarah',
  'start': 11,
  'end': 16},
 {'entity': 'Person_Name',
  'score': 0.8314036130905151,
  'index': 5,
  'word': 'Jessica',
  'start': 17,
  'end': 24},
 {'entity': 'Person_Name',
  'score': 0.849219560623169,
  'index': 6,
  'word': 'Parker',
  'start': 25,
  'end': 31},
 {'entity': 'Person_Name',
  'score': 0.7736506462097168,
  'index': 12,
  'word': 'Jessica',
  'start': 52,
  'end': 59},
 {'entity': 'location',
  'score': 0.7681230306625366,
  'index': 17,
  'word': 'College',
  'start': 74,
  'end': 81},
 {'entity': 'location',
  'score': 0.6825906038284302,
  'index': 18,
  'word': 'Park',
  'start': 82,
  'end': 86}]

In [12]:
import boto3

client = boto3.client('sagemaker')
response = client.list_models(
)

for model in response['Models']:
    print(model['ModelName'])

huggingface-pytorch-inference-2024-01-10-17-43-22-630
huggingface-pytorch-inference-2024-01-10-15-15-15-481
huggingface-pytorch-inference-2024-01-10-15-14-37-014


In [14]:
import boto3
import sagemaker
from sagemaker.transformer import Transformer

#NOTE INCREASE ml.m5.xlarge for transform job usage within Amazon in quotas

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Define the model name (replace with your deployed model name)
model_name = 'huggingface-pytorch-inference-2024-01-10-15-15-15-481'

# Define input and output data format
input_data_format = 'text/plain'  # Adjust based on your model's expected input format
output_data_format = 'application/jsonlines'  # Adjust based on your model's output format

# Define S3 paths for input and output
input_s3_path = 's3://textsagemaker/input-data/'  # Replace with the correct S3 path
output_s3_path = 's3://outputbatchjobsagemaker/output-data/'  # Replace with the correct S3 path

# Initialize a transformer object
transformer = Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    strategy='SingleRecord'
)

# Start the batch transform job
transformer.transform(
    data=input_s3_path,
    data_type='S3Prefix',
    content_type=input_data_format,
    split_type='Line',
    input_filter='$[0,5]'
)

# Wait for the job to finish
transformer.wait()



INFO:sagemaker:Creating transform job with name: huggingface-pytorch-inference-2024-01-10-18-12-51-287


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTransformJob operation: The account-level service limit 'ml.m5.xlarge for transform job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

In [ ]:
import jsonlines

def extract_consecutive_entities(file_path, threshold=0.8):
    # Open the JSON lines file and extract consecutive entity types
    with jsonlines.open(file_path) as reader:
        consecutive_entities = {'PERSON': [], 'LOCATION': []}
        current_entity_type = None
        current_entity_score = None

        for line in reader:
            entities = line.get('entities', [])

            for entity in entities:
                entity_type = entity['type']
                entity_score = entity['score']

                if (
                    current_entity_type == entity_type
                    and current_entity_score > threshold
                    and entity_score > threshold
                ):
                    # Consecutive entity with the same type and score above threshold
                    consecutive_entities[entity_type].append(entity['text'])
                else:
                    # Start a new consecutive sequence
                    consecutive_entities[entity_type] = [entity['text']]

                current_entity_type = entity_type
                current_entity_score = entity_score

    return consecutive_entities

def process_output_files(bucket, prefix):
    # Create an S3 client
    s3_client = boto3.client('s3')

    # List objects in the S3 bucket with the specified prefix
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

    # Process each output file
    for obj in response.get('Contents', []):
        file_path = f"s3://{bucket}/{obj['Key']}"
        consecutive_entities = extract_consecutive_entities(file_path)

        print(f"Consecutive Entities extracted from {file_path}:")
        print("Person Names:", consecutive_entities['PERSON'])
        print("Locations:", consecutive_entities['LOCATION'])
        print("\n")

# Replace 'your-s3-bucket' and 'your-output-prefix' with your S3 bucket and output prefix
process_output_files(bucket='your-s3-bucket', prefix='your-output-prefix')
